In [1]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import simpy

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common
from tf_agents.trajectories import time_step as ts

#from env.RideSimulator.Grid import Grid

import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from RideSimulator.taxi_sim import run_simulation


In [2]:
#register custom env
import gym

gym.envs.register(
     id='taxi-v0',
     entry_point='env.taxi:TaxiEnv',
     max_episode_steps=1500,
     kwargs={'state_dict':None},
)

In [3]:
#hyper params

num_iterations = 200 # @param {type:"integer"}

initial_collect_steps = 1000  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}action

In [7]:
#load taxi env
env_name = "taxi-v0"
env = suite_gym.load(env_name)

tf_env = tf_py_environment.TFPyEnvironment(env)
env.reset()


TimeStep(step_type=array(0, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([ 0.,  0., 20.,  0.], dtype=float32))

In [5]:
#agent and policy
fc_layer_params = (100,)

"""
q_net = q_network.QNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    fc_layer_params=fc_layer_params)
taxi_sim
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()
"""

#random policy
policy = random_tf_policy.RandomTFPolicy(tf_env.time_step_spec(),
                                                tf_env.action_spec())

"""
#replay buffer
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffcollect_data_specer_capacity)
    
"""

'\n#replay buffer\nreplay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(\n    data_spec=agent.collect_data_spec,\n    batch_size=train_env.batch_size,\n    max_length=replay_buffcollect_data_specer_capacity)\n    \n'

In [6]:
run_simulation(policy)

[20000, 40000, 60000, 60000]
0 looking for driver at 0
0 looking for driver at 50
0 looking for driver at 100
0 looking for driver at 150
0 looking for driver at 200
0 got 0 at 200
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([55.,  0.,  0., 13.], dtype=float32))


AttributeError: 'tuple' object has no attribute 'rank'

In [7]:
ts.restart(np.array([0,0,0,0], dtype=np.int32))

TimeStep(step_type=array(0, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([0, 0, 0, 0], dtype=int32))

In [8]:
ts.transition(np.array([1,2,3,4], dtype=np.int32), reward=0.0, discount=1.0)

TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([1, 2, 3, 4], dtype=int32))

In [9]:
ts.termination(np.array([1,2,3,4], dtype=np.int32), reward=0.0)

TimeStep(step_type=array(2, dtype=int32), reward=array(0., dtype=float32), discount=array(0., dtype=float32), observation=array([1, 2, 3, 4], dtype=int32))

In [6]:
#train
num_iterations =1

for i in range (num_iterations):
    #run one episode of simulation and record states
    state_list = run_simulation(2)
    states = []
    actions = []
    
    #convert states directly to tf timesteps
    for i in range(len(state_list)):
        #create time step
        if i == 0:
            state_tf = ts.restart(np.array(state_list[i]["observation"], dtype=np.float32))
        if i == (len(state_list) - 1):
            state_tf = ts.termination(np.array(state_list[i]["observation"], dtype=np.float32), reward=state_list[i]["reward"])
        else:
            state_tf = ts.transition(np.array(state_list[i]["observation"], dtype=np.float32), reward=state_list[i]["reward"],  discount=1.0)
        
        #create action
        """if state_list[i]["action"] == 1:
            action = tf.constant([1], dtype=tf.int32)
        else:
            action = tf.constant([0], dtype=tf.int32)"""
        action = state_list[i]["action"]
            
        #print (state_tf)
        #print (action)
        states.append(state_tf)
        actions.append(action)
        
    trajectories = [] #TODO - use replay buffer for collection
    for j in range(len(states)-1):
        present_state = states[j]
        next_state = states[j+1]
        action = actions[j]
        traj = trajectory.from_transition(present_state, action, next_state)
        trajectories.append(traj)
        print(traj)
    
    
        
    """
    #re-register environemnt with new states
    env_name = 'taxi-v'+str(i)
    gym.envs.register(
         id=env_name,
         entry_point='env.taxi:TaxiEnv',
         max_episode_steps=1500,
         kwargs={'state_dict':state_list},
    )
    
    #reload new env
    env = suite_gym.load(env_name)
    tf_env = tf_py_environment.TFPyEnvironment(env)

    #reset tf env
    time_step = tf_env.reset()
    
    #loop through recorded steps
    for step in state_dict:
        present_state = tf_env.current_time_step()
        action = step.action
        new_state = tf_env.step(action)
        traj = trajectory.from_transition(time_step, action_step, next_time_step)
        replay_buffer.add_batch(traj)
    """
    
print(replay_buffer)

[20000, 40000, 60000, 60000]
0 looking for driver at 0
0 looking for driver at 50
0 looking for driver at 100
0 got 0 at 100
0 traveling to pickup point, [39 26] from [ 7 23]
driver 0 location updated  to  [22 36]
Trip distance: 20, time 132
0 finished the trip and released driver 0 at 152 to pool 5
Driver 0 finished trip
1 looking for driver at 200
1 looking for driver at 250
1 looking for driver at 300
1 looking for driver at 350
1 got 0 at 350
0 traveling to pickup point, [37  0] from [22 36]
driver 0 location updated  to  [13  7]
Trip distance: 25, time 389
1 finished the trip and released driver 0 at 414 to pool 3
Driver 0 finished trip
2 looking for driver at 500
3 looking for driver at 550
2 looking for driver at 550
3 looking for driver at 600
2 looking for driver at 600
2 got 0 at 600
0 traveling to pickup point, [56  4] from [13  7]
driver 0 location updated  to  [22 23]
Trip distance: 39, time 643
3 looking for driver at 650
2 finished the trip and released driver 0 at 682 t

53 looking for driver at 9250
52 looking for driver at 9250
49 looking for driver at 9250
54 got 0 at 9250
0 traveling to pickup point, [23  3] from [51  3]
driver 0 location updated  to  [16 11]
Trip distance: 11, time 9278
54 finished the trip and released driver 0 at 9289 to pool 4
Driver 0 finished trip
53 looking for driver at 9300
52 looking for driver at 9300
49 looking for driver at 9300
53 looking for driver at 9350
52 looking for driver at 9350
49 looking for driver at 9350
55 looking for driver at 9400
53 looking for driver at 9400
52 looking for driver at 9400
49 was cancelled!
55 looking for driver at 9450
53 looking for driver at 9450
52 looking for driver at 9450
55 got 0 at 9450
0 traveling to pickup point, [5 8] from [16 11]
driver 0 location updated  to  [43 28]
Trip distance: 43, time 9461
56 looking for driver at 9500
53 looking for driver at 9500
52 looking for driver at 9500
55 finished the trip and released driver 0 at 9504 to pool 7
Driver 0 finished trip
56 loo

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'action'

In [3]:
# startup simulation

def simpy_episode(rewards, steps, time_step, tf_env, policy):

    TIME_MULTIPLIER = 50
    DRIVER_COUNT = 1
    TRIP_COUNT = 8000
    RUN_TIME = 10000
    INTERVAL = 20
    # GRID_WIDTH = 3809
    # GRID_HEIGHT = 2622
    GRID_WIDTH = 60
    GRID_HEIGHT = 40
    HEX_AREA = 2.6

    Env = simpy.Environment()
    map_grid = Grid(env=Env, width=GRID_WIDTH, height=GRID_HEIGHT, interval=INTERVAL, num_drivers=DRIVER_COUNT,
                    hex_area=HEX_AREA)

    taxi_spots = map_grid.taxi_spots
    driver_list = create_drivers(Env, DRIVER_COUNT, map_grid)
    driver_pools = map_grid.driver_pools

    run_simulation(TRIP_COUNT, RUN_TIME, DRIVER_COUNT, TIME_MULTIPLIER, map_grid, taxi_spots, driver_list, driver_pools, Env, rewards, steps, time_step, tf_env, policy)
    t_count = 0
    for dr in driver_list:
        d_t_count = dr.total_trip_count
        t_count += d_t_count
        print(f"{dr.id} completed {d_t_count}")

    print(f"Total trip count: {t_count}")

In [5]:
var = tf.random.uniform([1], 0, 2, dtype=tf.int32)
var[0] = 2
print (var)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [10]:
#simple episode run - atttempt 1

time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
    simpy_episode(rewards, step, time_step, tf_env, policy)

    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 15)

In [ ]:
#simple episode run - atttempt 2

#time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
    time_step = tf_env.reset()
    
    
    
    
    
    
    simpy_episode(rewards, step, time_step, tf_env, policy)

    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

In [ ]:
#simple episode run template
"""
time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
  episode_reward = 0
  episode_steps = 0
  while not time_step.is_last():
    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

print('num_episodes:', num_episodes, 'num_steps:', num_steps)
print('avg_length', avg_length, 'avg_reward:', avg_reward)
"""